In [1]:
import os.path
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os


In [2]:
def get_cluster_file(_filename):
    # get the abs path of the directory of the code/script
    # Factors and Ridership Data\ code
    current_dir = pathlib.Path(__file__).parent.absolute()
    # Change the directory
    # \Model Estimation\Est4
    # print("current directory at get_cluster_file ",current_dir)
    current_dir = current_dir.parents[0] / 'Model Estimation' / 'Est6'
    os.chdir(str(current_dir))
    # print("set directory at get_upt_fac_file ", current_dir)
    df = pd.read_csv(_filename)
    return df 

In [3]:
def create_upt_fac_total_apta4_cluster(_filename, _clustervalue):
    # get the abs path of the directory of the code/script
    # Factors and Ridership Data\ code
    current_dir = pathlib.Path(__file__).parent.absolute()
    folder_name = chart_name = _filename.split('.')[0]
    # Change the directory
    # \Script Outputs \ Cluster_wise_summation_files
    # print("current directory at get_cluster_file ",current_dir)
    current_dir = current_dir.parents[0] / 'Model Estimation' / 'Est4'
    os.chdir(str(current_dir))
    # print("set directory at get_cluster_file ", current_dir)
    df_org = pd.read_csv(_filename)    
    # create cumulative column and update the column    
    # create new columns
    col_name = ['VRM_ADJ_log_FAC','FARE_per_UPT_2018_log_FAC','POP_EMP_log_FAC','GAS_PRICE_2018_log_FAC',
                'TOTAL_MED_INC_INDIV_2018_log_FAC', 'Tot_NonUSA_POP_pct_FAC', 'PCT_HH_NO_VEH_FAC', 'TSD_POP_PCT_FAC',
                'JTW_HOME_PCT_FAC','RAIL_COMPETITION_FAC', 'YEARS_SINCE_TNC_BUS_FAC', 'YEARS_SINCE_TNC_RAIL_FAC',
                'BIKE_SHARE_BUS_FAC','scooter_flag_BUS_FAC','BIKE_SHARE_RAIL_FAC','Total_FAC']
    cum_col = []
    col_UPT_ADJ = ['UPT_ADJ']

    for col in col_name:
        df_org[str(col)+'_cumsum'] = df_org[col]
        cum_col.append(str(col)+'_cumsum')
    
    cluster_values =_clustervalue

    # # for each cluster_id get the cumulative addition starting from 2002-->2018
    # os.chdir(output_folder)
    for col in cum_col:
        df_org[col] = df_org.groupby([cluster_values,'RAIL_FLAG'])[col].cumsum()

    # # create a new column which is diff between UPT_ADJ - CUMSUM colmn
    for col in cum_col:
         df_org['UPT_ADJ_' + str(col)] = df_org['UPT_ADJ'] - df_org[col]        
    
    # save the cumulative file as UPT_filename.csv
    df_org.to_csv("UPT_"+folder_name+'.csv')
    print ("Success")
    
    # prepare_charts(df_org,clusters_col,chartinitials,file_name)
    

In [4]:
def prepare_charts(_df, _clustercolumn, _charinitials,_file_name):
    df = _df
    clustercolumn = _clustercolumn
    chartinitials = _charinitials
    folder_path = _file_name
    # get unique clusters
    clusters = df[clustercolumn].unique()
    clusters.sort()
    # get unique modes
    df.rename(columns={'RAIL_FLAG': 'Mode'}, inplace=True)
    modes = df['Mode'].unique()
    modes.sort()
    # get unique years
    years = df['Year'].unique()
    years.sort()      

    for cluster in clusters:        
        df_fltr = df[df[clustercolumn] == cluster]
        # Print the cluster 
        col_index = df_fltr.columns.get_loc(clustercolumn)
        cluster_code = str(df_fltr.iloc[0, col_index])
        print('Cluster Code:' + str(cluster_code))
        df_fltr['Year'] = pd.to_datetime(df_fltr['Year'].astype(str), format='%Y')
        df_fltr_mod = df_fltr.set_index(pd.DatetimeIndex(df_fltr['Year']).year)
        
        for mode in modes:            
            if mode == 0:
                mode_name = "BUS"
            else:
                mode_name = "RAIL"                
            # get number of sub-plots defined - 4*2 means 4 rows having 2 graphs (each sized 18x9) in each row = 8 graphs
            fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(18, 9), constrained_layout=True)        
            x = 1 
                           
            df_fltr_mode = df_fltr_mod[df_fltr_mod.Mode == mode]                          
            # Year vs UPT_ADJ_Total_FAC_cumsum --> Graph (0,0)
            # df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ_Total_FAC_cumsum', 
            #                                   label='Hypothezized rdrship if no changes in total FACs', 
            #                                   ax=ax[0][0], legend=True)
            # df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ', label='Observed Rdrship', ax=ax[0][0],
            #                                   legend=True, color='black', linewidth=2)
            # # Paint the area
            # ax[0][0].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_Total_FAC_cumsum'].values,
            #                       df_fltr_mode['UPT_ADJ'].values,
            #                       where=df_fltr_mode['UPT_ADJ'].values>df_fltr_mode['UPT_ADJ_Total_FAC_cumsum'].values,
            #                       facecolor='red', interpolate=True, alpha=0.1)
            # ax[0][0].fill_between(df_fltr_mode['Year'].values, df_fltr_mode['UPT_ADJ_Total_FAC_cumsum'].values,
            #                       df_fltr_mode['UPT_ADJ'].values,
            #                       where=df_fltr_mode['UPT_ADJ'].values<=df_fltr_mode['UPT_ADJ_Total_FAC_cumsum'].values, 
            #                       facecolor='green', interpolate=True, alpha=0.1)
            # ax[0][0].set(xlabel="Years", ylabel='Ridership')
            # ax[0][0].legend(loc='best')
            # ax[0][0].set_autoscaley_on(False)
            # try:                                
            #     ax[0][0].grid(True)
            #     ax[0][0].margins(0.20)
            #     ax[0][0].set_ylim([0,max(df_fltr_mode[['UPT_ADJ', 'UPT_ADJ_Total_FAC_cumsum']].values.max(1))])
            # except ValueError:
            #     pass
            
            # Year vs POP_EMP_log_FAC_cumsum --> Graph (1,0)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ_POP_EMP_log_FAC_cumsum', 
                                              label='Hypothezized rdrship if no changes in population & employment', 
                                              ax=ax[0][0], legend=True)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ', label='Observed Rdrship', ax=ax[0][0], 
                                              legend=True, color='black', linewidth=2)
            # Paint the area
            ax[0][0].fill_between(df_fltr_mode['Year'].values, df_fltr_mode['UPT_ADJ_POP_EMP_log_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values, 
                                  where=df_fltr_mode['UPT_ADJ'].values>df_fltr_mode['UPT_ADJ_POP_EMP_log_FAC_cumsum'].values,
                                  facecolor='red', interpolate=True, alpha=0.1)
            ax[0][0].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_POP_EMP_log_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values,  
                                  where=df_fltr_mode['UPT_ADJ'].values<=df_fltr_mode['UPT_ADJ_POP_EMP_log_FAC_cumsum'].values, 
                                  facecolor='green', interpolate=True, alpha=0.1)     
            ax[0][0].set(xlabel="Years", ylabel='Ridership')
            ax[0][0].legend(loc='best')
            # ax[1][0].set_autoscaley_on(False)
            try:                
                ax[0][0].grid(True)
                ax[0][0].margins(0.20)
                ax[0][0].set_ylim([0,max(df_fltr_mode[['UPT_ADJ', 'UPT_ADJ_POP_EMP_log_FAC_cumsum']].values.max(1))])
            except ValueError:
                pass


            # Year vs UPT_ADJ_TSD_POP_PCT_FAC_cumsum --> Graph (2,0)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ_TSD_POP_PCT_FAC_cumsum', 
                                              label='Hypothezized rdrship if no change in TSD Pop', ax=ax[1][0], 
                                              legend=True)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ', label='Observed Rdrship', legend=True, 
                                              ax=ax[1][0], color='black', linewidth=2)
            # Paint the area
            ax[1][0].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_TSD_POP_PCT_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values, 
                                  where=df_fltr_mode['UPT_ADJ'].values>df_fltr_mode['UPT_ADJ_TSD_POP_PCT_FAC_cumsum'].values, 
                                  facecolor='red', interpolate=True, alpha=0.1)
            ax[1][0].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_TSD_POP_PCT_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values,
                                  where=df_fltr_mode['UPT_ADJ'].values<=df_fltr_mode['UPT_ADJ_TSD_POP_PCT_FAC_cumsum'].values,
                                  facecolor='green', interpolate=True, alpha=0.1)
            ax[1][0].set(xlabel="Years", ylabel='Ridership')
            ax[1][0].legend(loc='best')
            # ax[2][0].set_autoscaley_on(False)
            try:
                ax[1][0].grid(True)
                ax[1][0].margins(0.20)
                ax[1][0].set_ylim([0,max(df_fltr_mode[['UPT_ADJ', 'UPT_ADJ_TSD_POP_PCT_FAC_cumsum']].values.max(1))])
            except ValueError:
                pass

            # Year vs UPT_ADJ_PCT_HH_NO_VEH_FAC_cumsum --> Graph (3,0)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ_PCT_HH_NO_VEH_FAC_cumsum', 
                                              label='Hypothezized rdrship if no change in PCT HH NO VEH', 
                                              ax=ax[2][0],legend=True)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ', label='Observed Rdrship', ax=ax[2][0], legend=True,
                                              color='black', linewidth=2)
            # Paint the area
            ax[2][0].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_PCT_HH_NO_VEH_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values, 
                                  where=df_fltr_mode['UPT_ADJ'].values>df_fltr_mode['UPT_ADJ_PCT_HH_NO_VEH_FAC_cumsum'].values, 
                                  facecolor='red', interpolate=True, alpha=0.1)
            ax[2][0].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_PCT_HH_NO_VEH_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values,
                                   where=df_fltr_mode['UPT_ADJ'].values<=df_fltr_mode['UPT_ADJ_PCT_HH_NO_VEH_FAC_cumsum'].values, 
                                  facecolor='green', interpolate=True, alpha=0.1)
            ax[2][0].set(xlabel="Years", ylabel='Ridership')
            ax[2][0].legend(loc='best')
            # ax[3][0].set_autoscaley_on(False)
            try:               
                ax[2][0].grid(True)
                ax[2][0].margins(0.20)
                ax[2][0].set_ylim([0,max(df_fltr_mode[['UPT_ADJ', 'UPT_ADJ_PCT_HH_NO_VEH_FAC_cumsum']].values.max(1))])
            except ValueError: 
                pass

            # Year vs UPT_ADJ_VRM_ADJ_log_FAC_cumsum --> Graph (0,1)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ_VRM_ADJ_log_FAC_cumsum', 
                                              label='Hypothezized rdrship if no change in VRMs', 
                                              ax=ax[0][1], legend=True)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ', label='Observed Rdrship', ax=ax[0][1], legend=True,
                                              color='black', linewidth=2)
            # Paint the area
            ax[0][1].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_VRM_ADJ_log_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values,
                                  where=df_fltr_mode['UPT_ADJ'].values>df_fltr_mode['UPT_ADJ_VRM_ADJ_log_FAC_cumsum'].values, 
                                  facecolor='red', interpolate=True, alpha=0.1)
            ax[0][1].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_VRM_ADJ_log_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values, 
                                   where=df_fltr_mode['UPT_ADJ'].values<=df_fltr_mode['UPT_ADJ_VRM_ADJ_log_FAC_cumsum'].values, 
                                  facecolor='green', interpolate=True, alpha=0.1)
            ax[0][1].set(xlabel="Years", ylabel='Ridership')
            ax[0][1].legend(loc='best')
            # ax[0][1].set_autoscaley_on(False)
            try:                                              
                ax[0][1].grid(True)
                ax[0][1].margins(0.20)
                ax[0][1].set_ylim([0,max(df_fltr_mode[['UPT_ADJ', 'UPT_ADJ_VRM_ADJ_log_FAC_cumsum']].values.max(1))])
            except ValueError: 
                pass
            
            # Year vs UPT_ADJ_GasPrice_log_FAC_cumsum --> Graph (1,1)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ_GasPrice_log_FAC_cumsum',
                                              label='Hypothezized rdrship if no change in Gas Prices', 
                                              ax=ax[1][1], legend=True)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ', label='Observed Rdrship', ax=ax[1][1], legend=True, 
                                              color='black', linewidth=2)
            # Paint the area
            ax[1][1].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_GasPrice_log_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values,  
                                  where=df_fltr_mode['UPT_ADJ'].values>df_fltr_mode['UPT_ADJ_GasPrice_log_FAC_cumsum'].values, 
                                  facecolor='red', interpolate=True, alpha=0.1)
            ax[1][1].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_GasPrice_log_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values,  
                                   where=df_fltr_mode['UPT_ADJ'].values<=df_fltr_mode['UPT_ADJ_GasPrice_log_FAC_cumsum'].values, 
                                  facecolor='green', interpolate=True, alpha=0.1)
            ax[1][1].set(xlabel="Years", ylabel='Ridership')
            ax[1][1].legend(loc='best')
            # ax[1][1].set_autoscaley_on(False)
            try:                
                ax[1][1].grid(True)
                ax[1][1].margins(0.20)
                ax[1][1].set_ylim([0,max(df_fltr_mode[['UPT_ADJ', 'UPT_ADJ_GasPrice_log_FAC_cumsum']].values.max(1))])
            except ValueError:
                pass
            
            # Year vs UPT_ADJ_FARE_per_UPT_log_FAC_cumsum --> Graph (2,1)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ_FARE_per_UPT_log_FAC_cumsum', 
                                              label='Hypothezized rdrship if no change in UPTs',
                                              ax=ax[2][1], legend=True)
            df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ', label='Observed Rdrship', ax=ax[2][1], 
                                              legend=True, color='black', linewidth=2)
            # Paint the area
            ax[2][1].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_FARE_per_UPT_log_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values,  
                                  df_fltr_mode['UPT_ADJ'].values>df_fltr_mode['UPT_ADJ_FARE_per_UPT_log_FAC_cumsum'].values, 
                                  facecolor='red', interpolate=True, alpha=0.1)
            ax[2][1].fill_between(df_fltr_mode['Year'].values,df_fltr_mode['UPT_ADJ_FARE_per_UPT_log_FAC_cumsum'].values,
                                  df_fltr_mode['UPT_ADJ'].values,  
                                  df_fltr_mode['UPT_ADJ'].values<=df_fltr_mode['UPT_ADJ_FARE_per_UPT_log_FAC_cumsum'].values, 
                                  facecolor='green', interpolate=True, alpha=0.1)
            ax[2][1].set(xlabel="Years", ylabel='Ridership')        
            ax[2][1].legend(loc='best')
            # ax[2][1].set_autoscaley_on(False)
            try:                
                ax[2][1].grid(True)
                ax[2][1].margins(0.20)
                ax[2][1].set_ylim([0,max(df_fltr_mode[['UPT_ADJ', 'UPT_ADJ_FARE_per_UPT_log_FAC_cumsum']].values.max(1))])
            except ValueError:
                pass
#             # Year vs Total_FAC_Scaled --> Graph (3,1)
#             df_fltr_mode.groupby('Mode').plot(x='Year', y='TSD_POP_PCT_FAC_cumsum', label='UPT_ADJ - TSD_POP_PCT_FAC_cumsum', ax=ax[3][1], legend=True, marker='',color='skyblue',linewidth=2)
#             df_fltr_mode.groupby('Mode').plot(x='Year', y='UPT_ADJ', label='UPT_ADJ', ax=ax[3][1], legend=True, marker='',color='olive',linewidth=2)
#             ax[3][1].set(xlabel="Years", ylabel='TSD_POP_PCT')
#             ax[3][1].legend(loc='best')
            fig.suptitle(('Cluster Code:' + str(cluster_code) + "-" + str(mode)),fontsize=14)
            fig.tight_layout()
            _figno = x
            # code to let these file save in the specific folder
            os.chdir(output_folder)
            # add folder name
    #         save_folder = output_folder +'\\' + folder_path
            os.path.join(output_folder, folder_path)
            
            if not os.path.exists(os.path.join(output_folder, folder_path)):
                os.mkdir(folder_path)
                print(folder_path + " for " + mode_name+" : sucessfully created")
            else:
                print (folder_path + " for " + mode_name+" : already exists")
            
            mod = output_folder + "\\" + str(folder_path)
            
            os.chdir(mod)
            fig.savefig(("Fig "+ str(_figno) + "-" + cluster_code + " - " + mode_name + ".png"))
            plt.suptitle(cluster_code,fontsize=14)
            plt.close(fig)
            x += 1  

In [5]:
def main():
    # Pass on the cluster_file and cluster_column
    create_upt_fac_total_apta4_cluster("FAC_totals_APTA4_CLUSTERS.csv","CLUSTER_APTA4")

if __name__ == "__main__":
    main()

NameError: name '__file__' is not defined

In [ ]:
# # libraries and data
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
#  
# # Make a data frame
# df=pd.DataFrame({'x': range(1,11), 'y1': np.random.randn(10), 'y2': np.random.randn(10)+range(1,11), 'y3': np.random.randn(10)+range(11,21), 'y4': np.random.randn(10)+range(6,16), 'y5': np.random.randn(10)+range(4,14)+(0,0,0,0,0,0,0,-3,-8,-6), 'y6': np.random.randn(10)+range(2,12), 'y7': np.random.randn(10)+range(5,15), 'y8': np.random.randn(10)+range(4,14), 'y9': np.random.randn(10)+range(4,14) })
#  
# # Initialize the figure
# plt.style.use('seaborn-darkgrid')
#  
# # create a color palette
# palette = plt.get_cmap('Set1')
#  
# # multiple line plot
# num=0
# for column in df.drop('x', axis=1):
#     num+=1
#  
#     # Find the right spot on the plot
#     plt.subplot(3,3, num)
#  
#     # plot every groups, but discreet
#     for v in df.drop('x', axis=1):
#         plt.plot(df['x'], df[v], marker='', color='grey', linewidth=0.6, alpha=0.3)
#  
#     # Plot the lineplot
#     plt.plot(df['x'], df[column], marker='', color=palette(num), linewidth=2.4, alpha=0.9, label=column)
#  
#     # Same limits for everybody!
#     plt.xlim(0,10)
#     plt.ylim(-2,22)
#  
#     # Not ticks everywhere
#     if num in range(7) :
#         plt.tick_params(labelbottom='off')
#     if num not in [1,4,7] :
#         plt.tick_params(labelleft='off')
#  
#     # Add title
#     plt.title(column, loc='left', fontsize=12, fontweight=0, color=palette(num) )
#  
# # general title
# plt.suptitle("How the 9 students improved\nthese past few days?", fontsize=13, fontweight=0, color='black', style='italic', y=1.02)
#  
# # Axis title
# plt.text(0.5, 0.02, 'Time', ha='center', va='center')
# plt.text(0.06, 0.5, 'Note', ha='center', va='center', rotation='vertical')